**Installed Package**

In [1]:
%pip install opencv-python-headless
%pip install mediapipe
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**Libraries**

In [2]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
import time

**Code Structure**

In [23]:

# Initialize mediapipe pose and drawing utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Angle calculation
def calculateAngle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180.0 else angle

# Draw angle on joint (Optional to show the Angles in the image)
def draw_angle(image, angle, landmark):
    h, w, _ = image.shape
    cx, cy = int(landmark.x * w), int(landmark.y * h)
    cv2.putText(image, f'{int(angle)} deg', (cx - 20, cy - 20),
                cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 2)


# Pose classification and angle display
def classifyPose(landmarks, output_image):
    label = 'Unknown Pose'
    color = (0, 0, 255)

    # Calculate angles
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])


    # Pose classification logic
    if 40 < left_elbow_angle < 60:
        if 15 < left_shoulder_angle < 25:
            label = 'Push-Up Down Position'
    elif 165 < left_elbow_angle < 180:
        if 70 < left_shoulder_angle < 80:
            label = 'Push-Up Start Position'

    if label != 'Unknown Pose':
        color = (0, 255, 0)

    cv2.putText(output_image, label, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)

    return output_image, label

# ✅ Webcam setup (index 0 If you use a WebCam)
cap = cv2.VideoCapture(0)

# Push-up counter variables
counter = 0
stage = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        frame, label = classifyPose(landmarks, frame)

        # Push-up logic
        if label == 'Push-Up Down Position':
            stage = 'down'
        elif label == 'Push-Up Start Position' and stage == 'down':
            stage = 'up'
            counter += 1

        # Show counter
        cv2.putText(frame, f'Push-Ups: {counter}', (10, 70),
                    cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 0), 2)

    # Display the frame inline in Jupyter Notebook
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # Add a small delay to simulate frame rate and allow interruption
    time.sleep(0.02)  # Adjust the delay as needed for smoother display
    from IPython.display import clear_output
    clear_output(wait=True)

cap.release()
